In [ ]:
import os
import datetime
import dateutil
import tarfile
import requests
import pandas as pd
import pandas.io.sql as sqlio

%matplotlib inline

On dispose de tableaux sauvés au format Pandas qui représentent les cours des actions à un moment donnée sur un marché donné. Ces deux informations sont dans le nom du fichier.

On commence par extraire les sauvegardes de l'archive puis on regarde la première sauvegarde que l'on a du compartiment A de la bourse de Paris.

In [ ]:
!mkdir tmp
!rm -rf tmp/2018
stream = requests.get('https://www.lrde.epita.fr/~ricou/boursorama2018.tgz', stream=True)
tarfile.open(fileobj=stream.raw, mode='r|gz').extractall('tmp/') # try 'r:gz' if there is an error

In [ ]:
os.listdir('tmp/2018')[:5]

In [ ]:
df = pd.read_pickle('tmp/2018/compA 2018-05-16 11:05:02.385909') # read_pickle to read Pandas DataFrames

In [ ]:
df

## Création de notre super DataFrame

On va stocker les valeurs de toutes les actions du compartiment A de 2018 dans un DataFrame en intrégrant la date (et heure) dans l'index (ce qui fait faire un DataFrame à 2 index). Attention on a extrait aussi d'autres marchés, il faut donc ne choisir que les fichiers qui contiennent compA (`find`  de String).

En utilisant la concaténation, créer ce DataFrame multi-index qu'on appelle `compA` (cela peut être fait en une commande). Affichez compA et indiquez un premier problème.

In [ ]:
import glob


## Extraction de l'action AXA

Le but est d'extraire le cours de l'action d'AXA et de le tracer.

* Créez un DataFrame du nom d'`axa` qui contient toutes les informations sur AXA le long de l'année
* Transformez `axa` pour qu'il n'ait qu'un seul index (la date). Pour cela `reset_index` devrait vous aider.
* Calculez la moyenne du cours d'AXA. Vous devriez avoir un problème, corrigez le dataframe avec `astype` pour que cela marche.
* Tracer la courbe du cours d'AXA (on fera attention à bien avoir des dates en abscisse et qu'on couvre tout 2018). Si les dates ne sont pas lisibles, c'est que le type est mauvais.

## Un gros calcul

Remettre en état le dataframe `compA` afin d'avoir comme index de niveau 0 les symboles des entreprises. 
Vérifier que le résultat est bon en vérifiant que le cours d'AXA ressemble à celui que vous venez de tracer.

Attention, le volume indique le nombre d'actions échangées depuis le début de la journée. Il y a aussi des données fausses à corriger dans les volumes.
On prendra en compte qu'il y a des jours de congé de la bourse que notre base n'a pas pris en compte :
`day_off = ['2018-01-01', '2018-03-30', '2018-04-02', '2018-05-01', '2018-12-25', '2018-12-26']`

Calculez la masse monétaire échangée dans le compartiment A en 2018 (le résultat est 1,2 billion d'euros).

Fonction utiles pour cette partie : `apply`, `drop`, `reset_index`, `set_index`, `swaplevel`, `rename_axis`, `sort_index`,
`dropna`, `fillna`, `diff`, `dateutil.parser.parse`, `timedelta`

La bourse de la Londres échange plus d'un billiard par an.

## Fusion de données

On dispose d'une seconde sources de cours de bourse mais avec un décalage de 15 minutes sur la première source :

* Boursorama : l'heure du fichier est l'heure de la cotation
* Netfonds : l'heure du fichier est la cotation d'il y a 15 mn

Avec ces deux sources créez le dataframe `bourse` le plus juste possible avec une cotation toutes les 10 mn de 9h05 à 17h55 chaque jour de cotation (lorsqu'on a une seule donnée on la prend, lorsqu'on en a deux, on fait la moyenne).

Afficher le cours d'AXA pour l'année.

Le fichier `data/symboles.csv` permet de faire la correspondance entre les actions des sources. On ne s'intéresse qu'aux actions dont on a le symbole dans les deux sources.

__Attention__ La taille des données `euronx` de la seconde source s'approche des 15 GO ce que votre machine risque de 
ne pas aimer. Il est donc préférable de charger les données mois par mois.

#### Conseils 

* Créer `month_b` et `month_n` qui contiennent toutes les cotations d'un mois choisi, disons janvier pour tester
* Tracer la courbe d'AXA pour voir
* Faire de sorte que ces 2 tableaux aient exactement le même index (mêmes entreprises au niveau 0, même dates au niveau 1)
* Travaillez avec un seul titre dans vos 2 tableaux pour deboguer, AXA par exemple :)
* Pour la fusion un `Grouper` est utile.

In [ ]:
!rm -rf tmp/2018   # on retire les fichiers de boursorama
stream = requests.get('https://www.lrde.epita.fr/~ricou/netfonds2018.tar', stream=True)
tarfile.open(fileobj=stream.raw, mode='r|').extractall('tmp/')

In [ ]:
df = pd.read_pickle("tmp/2018/euronx 2018-04-06 18:00:02.517981.bz2")
df

In [ ]:
# on ne charge que le mois de janvier 
euronx = 

## La petite touche finale

On récupère sur les cours journaliers des entreprises d'Euronext sur [Quandl](https://www.quandl.com/) (l'interface Quandl de Pandas datareader 
est boguée aussi on utilisera l'API de Quandl).

Les symboles pour obtenir un cours sont `'EURONEXT/xx'` avec `xx` la valeur de la cotation extraite de la colonne Netfonds de la table de symbole en retirant les 2 premiers caractère et le dernier. Ainsi `E-CSP` devient `CS`.

* Créer un tableau `q2018` avec toutes les données de Quandl pour 2018 des entreprises que vous avez dans `bourse`
* Afficher `Low` et `High` de 2018 pour AXA sur une même courbe (en une commande)
* Ajouter à `bourse` à 9h00 le cours d'ouverture donné par Quandl et à 18h00 celui de fermeture
* Vérifier que les cours de `bourse` sont bien compris entre `Low` et `High` de `q2018`. Quel est le pourcentage de cas problématiques. Pour les cas problématiques à moins de 1 % du min ou max remplacer par le min ou le max. Pour les autres mettre un `np.nan` (pour cette partie il est utile de remettre le tableau à plat c.a.d. sans index).

Conseil : commencer par faire une sauvegarde de `bourse` sur le disque avec `to_pickle` pour ne pas devoir le recalculer en cas de problème. Garder dans une variable à part le résultat de la requête auprès de Quandl (il doit limiter le nombre d'interrogations par jour).

In [ ]:
import quandl

quandl.ApiConfig.api_key = "xxxxxx"  # inscrivez-vous sur Quandl pour avoir une clef